# MODEL TRAINING

In [1]:
import os
%pwd

"c:\\Users\\soulo\\MACHINE_LEARNING_UNIVERSE\\PROJECT'S\\End-to-End-Machine-Leaning-Project-With-Mlow\\research"

In [2]:
os.chdir('../')
%pwd

"c:\\Users\\soulo\\MACHINE_LEARNING_UNIVERSE\\PROJECT'S\\End-to-End-Machine-Leaning-Project-With-Mlow"

In [ ]:
# 1. config.yaml

## Entity

In [32]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

## Config Manager

In [28]:
from mlproject.constants import *
from mlproject.utils.common import create_directories, read_yaml


class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, parmas_filepath = PARAMS_FILE_PATH,schema_filepath = SCHEMA_FILE_PATH ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(parmas_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.config.TARGET_COLUMN  
        create_directories([config.root_dir])
        
        model_trainer_config=ModelTrainerConfig(
            
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name = config.model_name,
            alpha=params.aplha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name,
        )
        return model_trainer_config


## components

In [29]:
import os
import joblib
import pandas as pd
from mlproject import logger
from sklearn.linear_model import ElasticNet

In [30]:
class ModelTrainer:
    def __init__(self , config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column] , axis=1)
        test_x = test_data.drop([self.config.target_column] , axis=1)
        
        train_y = train_data([self.config.target_column])
        test_y = test_data([self.config.target_column])
        
        lr = ElasticNet(alpha=self.config.alpha , l1_ratio=self.config.l1_ratio , random_state=self.config.random_state)
        
        lr.fit(train_x , train_y)
        
        joblib.dump(lr , os.path.join(self.config.root_dir , self.config.model_name))

## Pipeline

In [31]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e :
    raise e 

[2023-11-25 12:33:06,159] : INFO : common :  yaml file: config\config.yaml loaded successfully]
[2023-11-25 12:33:06,161] : INFO : common :  yaml file: params.yaml loaded successfully]
[2023-11-25 12:33:06,164] : INFO : common :  yaml file: schema.yaml loaded successfully]
[2023-11-25 12:33:06,167] : INFO : common :  created directory at: artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'target_column'"